In [1]:
import csv
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import gensim.parsing
import gensim.downloader as api
import spacy
from datetime import date

wv = gensim.downloader.load("word2vec-google-news-300")
nlp = spacy.load("en_core_web_sm")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
data1 = pd.read_csv('/content/drive/My Drive/21BCE0096/CSE206L/datasets/Disease Symptom Prediction/dataset.csv', delimiter=",")
data2 = pd.read_csv('/content/drive/My Drive/21BCE0096/CSE206L/datasets/Disease Symptom Prediction/symptom_Description.csv', delimiter=",")
data3 = pd.read_csv('/content/drive/My Drive/21BCE0096/CSE206L/datasets/Disease Symptom Prediction/Symptom-severity.csv', delimiter=",")
data4 = pd.read_csv('/content/drive/My Drive/21BCE0096/CSE206L/datasets/Disease Symptom Prediction/symptom_precaution.csv', delimiter=",")

df = pd.DataFrame(data1)
diseases = pd.DataFrame(data2)
symptoms = pd.DataFrame(data3)
precautions = pd.DataFrame(data4)

dataset = pd.DataFrame(columns = np.union1d(["Disease"], symptoms.Symptom))
for i in range(len(df)):
    row = df.loc[i,:]
    row = row[~pd.isnull(row)]
    row2 = pd.Series(0, index = dataset.columns)
    if (row["Disease"].rstrip() in diseases.values):
        index = diseases.index[diseases['Disease'] == row["Disease"].strip()]
        row2["Disease"] = index
        for symptom in row:
            row2[symptom.strip()] = 1
        dataset.loc[len(dataset)] = row2

for column in dataset:
    if ((dataset[column] == 0).all()):
        dataset = dataset.drop([column], axis = 1)

#dataset

In [4]:
def preprocess(string):
    result = string.replace("diarrhoea", "diarrhea")
    result = result.strip().replace("_", " ")
    result = " ".join([token.lemma_ for token in nlp(result)])
    result = gensim.parsing.preprocessing.remove_stopwords(result)
    result = [x.strip() for x in result.split()]
    return result

def find_similarity(s1, s2):
    result = 0
    for word1 in s1:
        max_similarity = 0
        for word2 in s2:
            if (word1 == word2):
                temp = 1
            elif ((word1 in wv.key_to_index) and (word2 in wv.key_to_index)):
                temp = wv.similarity(word1, word2)
            else:
                temp = 0
            if (temp > max_similarity):
                max_similarity = temp
        result += max_similarity
    result /= len(s1)
    return result

symptom_list = symptoms.Symptom
symptom_list = symptom_list.apply(lambda x: preprocess(x))

def get_symptoms(user_symptoms):
    result = []
    for user_symptom in user_symptoms.split(","):
        user_symptom = preprocess(user_symptom)
        most_similar = "cat"
        max_similarity = 0
        for symptom in symptom_list:
            temp = find_similarity(user_symptom, symptom)
            if (temp > max_similarity):
                max_similarity = temp
                most_similar = symptom
        for symptom in symptoms.Symptom:
            if (preprocess(symptom) == most_similar):
                result.append(symptom)
                break
    return result

In [5]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

target_label = "Disease"
target = dataset[target_label]
features = dataset.drop([target_label], axis=1)

def result(featurelist, model):
    X_train, X_test, y_train, y_test = train_test_split(featurelist, target, test_size = 0.2, shuffle = True, random_state = 7)
    model.fit(X_train.values, y_train.values)
    prediction = model.predict(X_test.values)
    print("Test result:", model.score(X_train.values, y_train.values))
    print("Train result:", model.score(X_test.values, y_test.values))
    print("Confusion Matrix:\n",confusion_matrix(prediction,y_test.values))

def train():
    target_label = "Disease"
    target = dataset[target_label]
    features = dataset.drop([target_label], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, shuffle = True, random_state = 7)
    model.fit(X_train.values, y_train.values)

model = BernoulliNB()

def disease_name(index):
    return diseases.iloc[index,0]

print(disease_name(dataset.loc[79,"Disease"]))
inp = dataset.loc[79]
inp = inp.drop(["Disease"])

train()
print(disease_name(model.predict([inp])[0]))

Diabetes
Diabetes


In [6]:
def run_interface():
  severity_scale = {1:'Normal', 2:'Normal', 3:'Effected', 4:'Effected', 5:'Effected', 6:'Severe', 7:'Severe'}

  print("Welcome, ABCXYZ!")
  user_symptoms = get_symptoms(input("Enter the symptoms: "))

  symptom_row = pd.Series(0, index = dataset.columns)
  symptom_row = symptom_row.to_frame().T
  symptom_entry = symptom_row.drop(["Disease"], axis = 1)

  severity = 1
  for symptom in user_symptoms:
      symptom = symptom.strip().replace(" ", "_")
      if (symptom.strip() in symptoms.values):
          symptom_entry[symptom] = 1
          temp = symptoms.loc[symptoms.Symptom == symptom].weight
          temp = int(temp.iloc[0])
          severity = max(severity, temp)

  prediction = model.predict(symptom_entry.values)[0]
  symptom_row["Disease"] = prediction
  disease = disease_name(prediction)

  print("-" * 50, "Patient Report", "-" * 50)
  print("Date:", date.today().strftime("%d/%m/%Y"))
  print("Patient ID: 7439802")
  print("Name: ABCXYZ")
  report_symptoms = ", ".join([s.strip().replace("_", " ") for s in user_symptoms])
  print("Symptoms reported:", report_symptoms)
  print("-" * 116)

  print("Diagnosis:", disease)
  print("Severity:", severity_scale[severity], "\n")
  print("You are advised to take the following measures:")
  should_alert = False
  actions = precautions.loc[precautions.Disease == disease]
  actions = actions[~pd.isnull(actions)]
  for i in range(4):
    action = actions["Precaution_" + str(i + 1)].to_string(header=False, index=False)
    if (action != 'NaN'):
      print("-", action)
      if ((severity > 5) or (action.lower().find('consult') != -1) or (action.lower().find('ambulance') != -1)):
        should_alert = True
  if (should_alert):
    print("\nThe hospital authority has been alerted. One of our healthcare professionals will be in touch with you soon.")
  print("-" * 116)
  return symptom_row


In [7]:
symptom_row = run_interface()
dataset = pd.concat([dataset, symptom_row], ignore_index = True)
train()
#dataset

Enter the symptoms: cramps, fatigue, weight gain, swollen legs
Welcome, ABCXYZ!
-------------------------------------------------- Patient Report --------------------------------------------------
Date: 12/04/2024
Patient ID: 7439802
Name: ABCXYZ
Symptoms reported: cramps, fatigue, weight gain, swollen legs
--------------------------------------------------------------------------------------------------------------------
Diagnosis: Varicose veins
Severity: Effected 

You are advised to take the following measures:
- lie down flat and raise the leg high
- use oinments
- use vein compression
- dont stand still for long
--------------------------------------------------------------------------------------------------------------------
